In [2]:
import nltk #; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

In [3]:
from sklearn.datasets import load_files

# for reproducibility
random_state = 0 

DATA_DIR = "./bbc/"
data = load_files(DATA_DIR, encoding="utf-8", decode_error="replace", random_state=random_state)
df = pd.DataFrame(list(zip(data['data'], data['target'])), columns=['text', 'label'])
df.head()

,text,label
0,Tate & Lyle boss bags top award\n\nTate & Lyle...,0
1,Halo 2 sells five million copies\n\nMicrosoft ...,4
2,MSPs hear renewed climate warning\n\nClimate c...,2
3,Pavey focuses on indoor success\n\nJo Pavey wi...,3
4,Tories reject rethink on axed MP\n\nSacked MP ...,2


## Data cleaning
We first tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether. We will use Gensim’s simple_preprocess() for this purpose. Additionally, we set deacc=True to remove the punctuations.

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df.text.values))

print(data_words[:1])

[['tate', 'lyle', 'boss', 'bags', 'top', 'award', 'tate', 'lyle', 'chief', 'executive', 'has', 'been', 'named', 'european', 'businessman', 'of', 'the', 'year', 'by', 'leading', 'business', 'magazine', 'iain', 'ferguson', 'was', 'awarded', 'the', 'title', 'by', 'us', 'publication', 'forbes', 'for', 'returning', 'one', 'of', 'the', 'uk', 'venerable', 'manufacturers', 'to', 'the', 'country', 'top', 'companies', 'the', 'sugar', 'group', 'had', 'been', 'absent', 'from', 'the', 'ftse', 'for', 'seven', 'years', 'until', 'mr', 'ferguson', 'helped', 'it', 'return', 'to', 'growth', 'tate', 'shares', 'have', 'leapt', 'this', 'year', 'boosted', 'by', 'firming', 'sugar', 'prices', 'and', 'sales', 'of', 'its', 'artificial', 'sweeteners', 'after', 'years', 'of', 'sagging', 'stock', 'price', 'and', 'seven', 'year', 'hiatus', 'from', 'the', 'ftse', 'one', 'of', 'britain', 'venerable', 'manufacturers', 'has', 'returned', 'to', 'the', 'vaunted', 'index', 'forbes', 'said', 'mr', 'ferguson', 'took', 'the',

Then we build bi-grams and tri-grams from the texts. Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['tate', 'lyle', 'boss', 'bags', 'top', 'award', 'tate', 'lyle', 'chief_executive', 'has', 'been', 'named', 'european', 'businessman', 'of', 'the', 'year', 'by', 'leading', 'business', 'magazine', 'iain', 'ferguson', 'was', 'awarded', 'the', 'title', 'by', 'us', 'publication', 'forbes', 'for', 'returning', 'one', 'of', 'the', 'uk', 'venerable', 'manufacturers', 'to', 'the', 'country', 'top', 'companies', 'the', 'sugar', 'group', 'had', 'been', 'absent', 'from', 'the', 'ftse', 'for', 'seven', 'years', 'until', 'mr', 'ferguson', 'helped', 'it', 'return', 'to', 'growth', 'tate', 'shares', 'have', 'leapt', 'this', 'year', 'boosted', 'by', 'firming', 'sugar', 'prices', 'and', 'sales', 'of', 'its', 'artificial', 'sweeteners', 'after', 'years', 'of', 'sagging', 'stock', 'price', 'and', 'seven', 'year', 'hiatus', 'from', 'the', 'ftse', 'one', 'of', 'britain', 'venerable', 'manufacturers', 'has', 'returned', 'to', 'the', 'vaunted', 'index', 'forbes', 'said', 'mr', 'ferguson', 'took', 'the', 'he

The bigrams model is ready. Then we remove stopwords, lemmatize the text and extract n-grams.

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['bag', 'award', 'tate', 'lyle', 'name', 'businessman', 'lead', 'magazine', 'award', 'title', 'publication', 'forbe', 'return', 'venerable', 'manufacturer', 'country', 'top', 'company', 'sugar', 'group', 'year', 'help', 'return', 'growth', 'tate', 'share', 'boost', 'firm', 'sugar', 'price', 'sale', 'artificial', 'sweetener', 'year', 'sag', 'stock', 'price', 'year', 'venerable', 'manufacturer', 'return', 'vaunted', 'index', 'forbes', 'say', 'take', 'company', 'spending', 'career', 'consumer', 'good', 'giant', 'tate', 'original', 'member', 'index', 'operate', 'factory', 'additional', 'production', 'facility', 'country', 'previous', 'winner', 'include', 'fred', 'goodwin', 'former', 'gent']]


## Building LDA
First, we build the inputs for LDA model, namely a dictionary(id2word) and a corpus. 

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 3), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 2), (46, 1), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 3)]]


Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [16]:
[id2word[i] for i in range(10)]

['additional',
 'artificial',
 'award',
 'bag',
 'boost',
 'businessman',
 'career',
 'company',
 'consumer',
 'country']

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('additional', 1),
  ('artificial', 1),
  ('award', 2),
  ('bag', 1),
  ('boost', 1),
  ('businessman', 1),
  ('career', 1),
  ('company', 2),
  ('consumer', 1),
  ('country', 2),
  ('facility', 1),
  ('factory', 1),
  ('firm', 1),
  ('forbe', 1),
  ('forbes', 1),
  ('former', 1),
  ('fred', 1),
  ('gent', 1),
  ('giant', 1),
  ('good', 1),
  ('goodwin', 1),
  ('group', 1),
  ('growth', 1),
  ('help', 1),
  ('include', 1),
  ('index', 2),
  ('lead', 1),
  ('lyle', 1),
  ('magazine', 1),
  ('manufacturer', 2),
  ('member', 1),
  ('name', 1),
  ('operate', 1),
  ('original', 1),
  ('previous', 1),
  ('price', 2),
  ('production', 1),
  ('publication', 1),
  ('return', 3),
  ('sag', 1),
  ('sale', 1),
  ('say', 1),
  ('share', 1),
  ('spending', 1),
  ('stock', 1),
  ('sugar', 2),
  ('sweetener', 1),
  ('take', 1),
  ('tate', 3),
  ('title', 1),
  ('top', 1),
  ('vaunted', 1),
  ('venerable', 2),
  ('winner', 1),
  ('year', 3)]]

Now we have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and beta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
print(lda_model)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

LdaModel(num_terms=15071, num_topics=5, decay=0.5, chunksize=100)
[(0,
  '0.023*"say" + 0.016*"company" + 0.015*"firm" + 0.013*"use" + 0.009*"user" + '
  '0.008*"site" + 0.007*"sale" + 0.007*"people" + 0.007*"net" + '
  '0.006*"service"'),
 (1,
  '0.017*"music" + 0.015*"people" + 0.014*"technology" + 0.013*"game" + '
  '0.012*"use" + 0.012*"say" + 0.009*"show" + 0.007*"new" + 0.007*"tv" + '
  '0.007*"industry"'),
 (2,
  '0.045*"say" + 0.019*"would" + 0.008*"people" + 0.007*"could" + 0.007*"tell" '
  '+ 0.006*"government" + 0.006*"make" + 0.006*"go" + 0.005*"party" + '
  '0.005*"claim"'),
 (3,
  '0.026*"say" + 0.018*"year" + 0.010*"rise" + 0.009*"government" + '
  '0.007*"economy" + 0.007*"figure" + 0.007*"report" + 0.007*"country" + '
  '0.007*"also" + 0.006*"fall"'),
 (4,
  '0.014*"year" + 0.013*"good" + 0.012*"film" + 0.011*"say" + 0.009*"go" + '
  '0.008*"play" + 0.008*"take" + 0.008*"win" + 0.008*"first" + 0.007*"also"')]


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/yg211/PycharmProjects/ScratchPad/venv_nlp/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.002397 -0.094318       1        1  29.148411
4      0.175686 -0.115825       2        1  25.719189
0     -0.140245  0.126519       3        1  17.388575
3     -0.159834 -0.099765       4        1  16.786442
1      0.121996  0.183389       5        1  10.957385, topic_info=     Category         Freq     Term        Total  loglift  logprob
12    Default   912.000000     firm   912.000000  30.0000  30.0000
7     Default  1023.000000  company  1023.000000  29.0000  29.0000
1232  Default   807.000000    music   807.000000  28.0000  28.0000
78    Default  1101.000000     game  1101.000000  27.0000  27.0000
208   Default  1517.000000      use  1517.000000  26.0000  26.0000
...       ...          ...      ...          ...      ...      ...
135    Topic5   230.053940     also  2049.959717   0.0239  -5.0879
606    Topic5   184.147568     work   852.452576   0.6788  -5.3104
131    Topic5   176.569214    world   832.531067   0.6604  -5.3525
412    Topic5   171.855728      get  1187.110352   0.2785  -5.3795
600    Topic5   160.616425     time  1298.479736   0.1212  -5.4471

[355 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1189      5  0.989791  accessible
378       1  0.994484      accuse
3007      2  0.997270       actor
56        1  0.356862         add
56        2  0.289467         add
...     ...       ...         ...
54        1  0.050592        year
54        2  0.451048        year
54        3  0.089426        year
54        4  0.371598        year
54        5  0.037409        year

[614 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 4, 2])